In [1]:
! pip install gradio peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 36.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.0 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5579 sha256=d630905bb9070b43d30bb5f82d81bcc6794b4ae6570f55cf95c7feeaf984d471
  Stored in directory: /root/.cache/pip/wheels/01/a6/d1/1c0828c304a4283b2c1639a09ad86f83d7c487ef34c6b4a1bf
Successfully built ffmpy


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [12]:
torch.set_default_device("cuda")

In [23]:
import gradio as gr
from transformers import AutoTokenizer
import re
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM
from transformers import BitsAndBytesConfig
import torch

config = PeftConfig.from_pretrained("mohamedemam/Arabic-meeting-summarization")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-3b",quantization_config=bnb_config)
model = PeftModel.from_pretrained(model, "mohamedemam/Arabic-meeting-summarization")
# Load the tokenizer and model
model_name ="bigscience/bloomz-3b"
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-3b")
model.eval()

# Function to generate questions and answers with configurable parameters
def generate_qa(context, temperature, top_p,num_seq,l_p, num_b):
    input_text = context
    input_ids = tokenizer(text=input_text, return_tensors='pt')
    
    # Generate with configurable parameters
    output = model.generate(
        **input_ids,
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=num_seq,
   
        max_new_tokens=60,
        num_beams=num_b,
        length_penalty=l_p,    
        do_sample=True,
        
    )
    #
    generated_text = tokenizer.batch_decode(output, skip_special_tokens=True)

    formatted_output = "\n\n".join(set(generated_text))
    return formatted_output
iface = gr.Interface(
    fn=generate_qa,
    inputs=[
        gr.inputs.Slider(minimum=0.0, maximum=5, default=2.1, step=0.01, label="Temperature"),
        gr.inputs.Slider(minimum=0.0, maximum=1, default=0.5, step=0.01, label="Top-p"),     
        gr.inputs.Slider(minimum=1, maximum=20, default=3, step=1, label="num of sequance"), 
        gr.inputs.Slider(minimum=0.01, maximum=5, default=3, step=.01, label="l_p")
     ,
        gr.inputs.Slider(minimum=1, maximum=20, default=3, step=1, label="num of beams"),


    ],
    outputs=gr.outputs.Textbox(label="Generated Output"),
    title="Question Generation and Answering",
    description="Select an example context, choose a recommended word, adjust temperature and top-p. The model will generate questions and answers.",
)
# Launch the interface
iface.launch()


KeyboardInterrupt



In [22]:
    a = tokenizer(text="as", return_tensors='pt')
    
    # Generate with configurable parameters
    output = model.generate(input_ids= a['input_ids'].cuda(),
        temperature=1.2,
        top_p=.3,
        num_return_sequences=2,
        max_new_tokens=60,
        num_beams=3,
        length_penalty=1,    
        do_sample=True,
        
    )

In [30]:
w=""
def generate_qa(context, temperature, top_p,num_seq,l_p, num_b):
    input_text = f"{context}"+f"\n\nSummarize the previous text in three sentences in arabic:\n\n"
    input_ids = tokenizer(text=input_text, return_tensors='pt')
    global w
    w=input_text
    # Generate with configurable parameters
    output = model.generate(
        input_ids=input_ids['input_ids'].cuda(),
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=num_seq,
   
        max_new_tokens=60,
        num_beams=num_b,
        length_penalty=l_p,    
        do_sample=True,
        
    )
    
    #
    generated_text = tokenizer.batch_decode(output, skip_special_tokens=True)

    formatted_output = "\n\n".join(set(generated_text))
    return formatted_output

In [31]:

iface = gr.Interface(
    fn=generate_qa,
    inputs=[
        "text",
        gr.inputs.Slider(minimum=0.0, maximum=5, default=2.1, step=0.01, label="Temperature"),
        gr.inputs.Slider(minimum=0.0, maximum=1, default=0.5, step=0.01, label="Top-p"),     
        gr.inputs.Slider(minimum=1, maximum=20, default=3, step=1, label="num of sequance"), 
        gr.inputs.Slider(minimum=0.01, maximum=5, default=3, step=.01, label="l_p")
     ,
        gr.inputs.Slider(minimum=1, maximum=20, default=3, step=1, label="num of beams"),


    ],
    outputs=gr.outputs.Textbox(label="Generated Output"),
    title="Question Generation and Answering",
    description="Select an example context, choose a recommended word, adjust temperature and top-p. The model will generate questions and answers.",
)
# Launch the interface
iface.launch()

/tmp/ipykernel_94/1303577419.py:5: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Slider(minimum=0.0, maximum=5, default=2.1, step=0.01, label="Temperature"),
/tmp/ipykernel_94/1303577419.py:5: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Slider(minimum=0.0, maximum=5, default=2.1, step=0.01, label="Temperature"),
/tmp/ipykernel_94/1303577419.py:6: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Slider(minimum=0.0, maximum=1, default=0.5, step=0.01, label="Top-p"),
/tmp/ipykernel_94/1303577419.py:6: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Slider(minimum=0.0, maximum=1, default=0.5, step=0.01, label="Top-p"),
/tmp/ipykernel_94/1303577419.py:7: G

Running on local URL:  http://127.0.0.1:7869
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://b2c8cfc5855ddb5f9b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [32]:
w

'مرحبًا، أنا ديفيد ومن المفترض أن أكون مصممًا صناعيًا. أم، لقد حصلت للتو على إعلان المشروع حول ماهية المشروع. تصميم جهاز التحكم عن بعد. هذا كل ما في الأمر، ولم أحصل على أي شيء آخر. هل حصلت على نفس الشيء؟ رائع. هناك الكثير من العتاد. تمام. لا أستطيع الرسم. أم. نعم. حسنًا على أية حال، لا أعرف، إنه أول حيوان يمكنني التفكير فيه من أعلى رأسي. أم. نعم. السبب الرئيسي هو أنني أشعر بالحساسية تجاه معظم الحيوانات. حساسية من فراء الحيوانات، لذلك كانت الأسماك خيارًا طبيعيًا. أم، نعم، وأنا أحب الحيتان نوعًا ما. يأتون ويذهبون ويأكلون كل شيء في الأفق. وهي غير ضارة تمامًا وخفيفة ومثيرة للاهتمام. الذيل كبير بعض الشيء، على ما أعتقد. إنه كلب بعد العشاء إذن. همم. ربما يكون هذا منطقيًا من وجهة نظر التصميم، لأنه لديك أحرف أكثر تعقيدًا مثل اللغات الأوروبية، فأنت بحاجة إلى المزيد من الأزرار. لذلك، ربما. همم. نعم. وتستمر في خسارتهم. العثور عليهم أمر مؤلم حقًا، كما تعلمون. أعني أنها عادة ما تكون صغيرة جدًا، أو عندما تريدها بشكل صحيح، تنزلق خلف الأريكة أو يتم ركلها تحت الطاولة. أنت تعرف. نعم. مم-هممم. أعتقد أن أح

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-samsum-dialogsum/arabic_dialog.csv


In [1]:
!pip install  -U bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.7 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install  -U git+https://github.com/huggingface/transformers.git 

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-18b1g5zp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-18b1g5zp
  Resolved https://github.com/huggingface/transformers.git to commit 44a0490d3c46e62134b3fc1f0609cbdf83e571da
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7662134 sha256=36a9b38770edeb9d3ceebf6f3a28bec5422137799c511d9407f56c77428f20ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-h_telic9/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0


In [3]:
!pip install  -U git+https://github.com/huggingface/peft.git
!pip install  -U git+https://github.com/huggingface/accelerate.git
!pip install  datasets

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-p7n3nssw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-p7n3nssw
  Resolved https://github.com/huggingface/peft.git to commit 93d0c03d5ba6b2a6b16b7ca887e740a67bc680f3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.6.0.dev0-py3-none-any.whl size=107812 sha256=d83f766d5ce6ca0c04f8ee7f90fd6585fddd40d4e1adb69ba2675e63c389cb43
  Stored in directory: /tmp/pip-ephem-wheel-cache-rkpsim6r/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.5.0
    Uninstalling peft-0.5.0:
      Successfully uninstalled peft-0.5.0
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-hq6ehyc7
  Running command git clon

In [36]:
import pandas as pd

In [37]:
data=pd.read_csv("/kaggle/input/arabic-samsum-dialogsum/arabic_dialog.csv")

In [44]:
data.iloc[1221,3]

'إميلي: 👻\n إميلي: مرحبا.هل تشعر بالرغبة في تناول العشاء في ناندو الليلة؟\n إميلي: يمكنني حجز طاولة\n باكو: بالتأكيد ، اذهب إليها\n مايكل: نعم من فضلك.أنا على وشك أخذ الأنبوب إلى المنزل الآن\n مايكل: أرغب في الاستحمام قبل الخروج ، هل يمكنك حجز الجدول ، على سبيل المثال ، 8 مساءً؟\n إميلي: بالتأكيد ، لا مشكلة\n مايكل: أراك قريبًا على أي حال\n إميلي: بارد\n باكو: 👌'

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-3b")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-3b",quantization_config=nf4_config,)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
! pip install peft

In [8]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=4, 
    lora_alpha=16, 
            target_modules=["query_key_value"],

    lora_dropout=0.05, 
    bias="none", 
    
    task_type="CAUSAL_LM"
)
#model = get_peft_model(model, config)

In [11]:
from peft import AutoPeftModel,PeftModel

In [12]:
a=PeftModel.from_pretrained(model,"/kaggle/input/blooma/results_modified/checkpoint-800")

In [13]:
data=pd.read_csv("/kaggle/input/arabic-samsum-dialogsum/arabic_dialog.csv")

In [14]:
from torch.utils.data import DataLoader, Dataset


In [15]:
"""class SummarizationDataset(Dataset):
    def __init__(self, tokenized_dataset):
        a = tokenizer( (data['dialogue']+data['summary']).tolist(), truncation=True, padding='max_length', return_tensors="pt", max_length=512)
        self.dataset=  [
            {
            
                            "input_ids":inputs,
                            "attention_mask": att,
                            "labels":inputs
                        } 
                            for inputs,att in zip(a['input_ids'],a['attention_mask'])
        ]
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]"""

'class SummarizationDataset(Dataset):\n    def __init__(self, tokenized_dataset):\n        a = tokenizer( (data[\'dialogue\']+data[\'summary\']).tolist(), truncation=True, padding=\'max_length\', return_tensors="pt", max_length=512)\n        self.dataset=  [\n            {\n            \n                            "input_ids":inputs,\n                            "attention_mask": att,\n                            "labels":inputs\n                        } \n                            for inputs,att in zip(a[\'input_ids\'],a[\'attention_mask\'])\n        ]\n    def __len__(self):\n        return len(self.dataset)\n\n    def __getitem__(self, idx):\n        return self.dataset[idx]'

In [153]:
class SummarizationDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.dialogue = tokenized_dataset['dialogue']
        self.summary = tokenized_dataset['summary']
        self.tokenized_dataset=tokenized_dataset
    def __len__(self):
        return len(self.tokenized_dataset)

    def __getitem__(self, idx):
        a=tokenizer(self.dialogue[idx] +f"\n\nSummarize the previous text in three sentences in arabic:\n\n", return_tensors="pt")
        return {
            
                        "input_ids": a["input_ids"].cuda(),
            "attention_mask": a["attention_mask"].cuda(),
        }

In [154]:
dataset_train=SummarizationDataset(data)

In [155]:
print(data.iloc[23051,3])
print(data.iloc[23051,4])

رينجو: ماذا أحصل على سيليا من أجل عيد الميلاد؟
 لين: ooooo صعب
 رينجو: إنها أختك تعرفها بشكل أفضل
 winton: إنه gf الخاص بك أنت تعرفها أقرب
 رينجو: أنت غير مفيد للغاية
 لين: احصل عليها ايكيا sth
 ringo: sth لا يبدو prmising جدا
 رينجو: أنا ضائع جدا
 لين: ماذا عن القتال إلى إيطاليا؟احصل على قذيفة واحدة رخيصة تكون سعيدة على أي حال
 رينجو: يا سوبر ثكس كثيرا !!!
لا يعرف رينجو ما يحصل على صديقته ، سيليا ، لعيد الميلاد.سيليا هي أخت وينتون.يقترح لين شيئًا من ايكيا أو رحلة رخيصة إلى إيطاليا.


In [136]:
w=a.merge_and_unload() 

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:205: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [178]:
w

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((2560,), eps=1e-05, elementwise_af

In [182]:
from huggingface_hub import notebook_login
notebook_login()

In [187]:
torch.save(w.state_dict(),"bloom_weights.pth")

In [189]:
w.push_to_hub("mohamedemam/Arabic-meeting-summarization")

NotImplementedError: You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported

In [185]:
huggingface_hub.("mohamedemam/Arabic-meeting-summarization")

NotImplementedError: You are calling `save_pretrained` on a 4-bit converted model. This is currently not supported

In [181]:
output_tokens = a.generate(**dataset_test[153], max_new_tokens=50,do_sample=True,num_return_sequences=4 , num_beams=6 ,temperature=.8, top_p=0.9
                          )
for i in tokenizer.batch_decode(output_tokens,skip_special_tokens=True):
            print(i)
            print("end")

الشخص 1: يا لها من بلوزة جذابة ، تبدو جميلًا جدًا في هذا الفستان الأزرق!
  الشخص 2: حقا؟شكرًا لك.اشتريتها في شارع سبرينغ أمس.
  الشخص 1: إنه لطيف للغاية.يسير على ما يرام مع تنورتك.
  الشخص 2: شكرا لك.من الجيد جدًا منك أن تقول ذلك.
  الشخص 1: ليس على الإطلاق.هذا الفستان يظهر بالتأكيد ذوقك الجيد.
  الشخص 2: هذا مجاملة قادمة منك.

Summarize the previous text in three sentences in arabic:

يطلب شخص من شخص آخر شراء فستان.
end
الشخص 1: يا لها من بلوزة جذابة ، تبدو جميلًا جدًا في هذا الفستان الأزرق!
  الشخص 2: حقا؟شكرًا لك.اشتريتها في شارع سبرينغ أمس.
  الشخص 1: إنه لطيف للغاية.يسير على ما يرام مع تنورتك.
  الشخص 2: شكرا لك.من الجيد جدًا منك أن تقول ذلك.
  الشخص 1: ليس على الإطلاق.هذا الفستان يظهر بالتأكيد ذوقك الجيد.
  الشخص 2: هذا مجاملة قادمة منك.

Summarize the previous text in three sentences in arabic:

يطلب شخص من شخص آخر شراء فستان له.
end
الشخص 1: يا لها من بلوزة جذابة ، تبدو جميلًا جدًا في هذا الفستان الأزرق!
  الشخص 2: حقا؟شكرًا لك.اشتريتها في شارع سبرينغ أمس.
  الشخص 1: إنه لطيف ل

In [157]:
output_tokens = a.generate(**dataset_train[23007], max_new_tokens=50,top_p=.4,do_sample=True,num_return_sequences=4 , num_beams=4,temperature =1.5)
for i in tokenizer.batch_decode(output_tokens,skip_special_tokens=True):
            print(i)
            print("end")

IndexError: list index out of range

In [125]:
from transformers import Trainer, TrainingArguments

In [126]:
from torch.utils.data import  random_split

In [156]:
dataset_train,dataset_valid,dataset_test=random_split(dataset_train,[.85,.075,.075])

In [ ]:
import torch

In [ ]:
model.enable_input_require_grads()

In [ ]:
import wandb
wandb.login(key="14459c516497ab76a78f7fc1278bfe60d301d250")

In [ ]:
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=26,
    save_steps=200,
    learning_rate=2e-5,
            fp16=True
,
    weight_decay=0.001,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)
trainer = Trainer(
    model=model,
    args=train_params,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid
)

In [ ]:
model.load("/kaggle/working/bloom_weights.pth")

In [ ]:
trainer.train()

In [ ]:
torch.save(model,'bloomz.pt')